# Abell 1367 Orphan Cloud and Nucleated Dwarf Image Depth

## Imports


In [ ]:
# Python Imports
from pathlib import Path

# Numerical Imports
import numpy as np

# Astropy Collab Imports
from astropy import units as u
from astropy.io import fits
from astropy.wcs import WCS
from astropy.table import QTable
from astropy.coordinates import SkyCoord
from astropy.convolution import convolve
from photutils.utils import calc_total_error, ImageDepth
from photutils.background import Background2D
from photutils.segmentation import SourceFinder, make_2dgaussian_kernel
from regions import Regions

# Plotting
import matplotlib.pyplot as plt

## Notebook Setup

In [ ]:
# Paths
IMG_DIR = Path('../Images/ProcessedImages/HST/DrizzledImages')
FOV_DIR = Path('../Images/ProcessedImages/HST/DS9/FOVs')
DEP_DIR = Path('../Images/ProcessedImages/HST/DS9/Depths')

# Filters
FILTERS = ['F275W', 'F475W', 'F814W']

In [ ]:
N_SIGMA = 5.0

## Load Data

In [ ]:
# Setup Variables
magLims, depth = {}, {}

# Loop through Filters & Load
for filter in FILTERS:

    # Open Image
    with fits.open(IMG_DIR / f'A1367OC-{filter}_drc.fits') as hduList:

        # Get the Header, Image, and WCS
        hdr = hduList['SCI'].header
        img = hduList['SCI'].data << u.electron/u.s
        wcs = WCS(hdr)

        # Get the Zero Point
        zp  = -2.5 * np.log10(hdr['PHOTFLAM']) - 21.1
        zp += 18.6921
        zp -= 5 * np.log10(hdr['PHOTPLAM'])
        zpt = u.Magnitude(zp, unit=u.mag(u.AB/img.unit))

    # Get the Coverage Mask
    coverageMask = Regions().read(FOV_DIR / f'{filter}-FOV.reg')[0].to_pixel(wcs).to_mask()
    coverageMask = ~coverageMask.to_image(img.shape, bool)
    badPixMask   = ~np.isfinite(img) & ~coverageMask

    # Get Background
    bkg = Background2D(
        img, 128, mask=badPixMask, coverage_mask=coverageMask
    )

    # Set Threshold and Kernel
    kernel = make_2dgaussian_kernel(3.5*5/3, 9)
    threshold = 1.25 * bkg.background_rms

    # Get the Background Subtracted Image & Convolve
    img -= bkg.background
    convData = convolve(
        img, kernel
    )

    # Find Sources
    finder = SourceFinder(10, nproc=4, deblend=False)
    segMap = finder(convData, threshold)
    srcMask = segMap.make_source_mask()

    # Get the Depth
    mask_pad = 20 if filter != 'F275W' else 0
    depth[filter] = ImageDepth(
        0.5/0.03, nsigma=N_SIGMA, mask_pad=mask_pad, napers=2000, niters=50,
        overlap=False, seed=None, zeropoint=zpt.value
    )
    _, magLims[filter] = depth[filter](img.value, coverageMask | badPixMask | srcMask)

In [ ]:
for filter, magLim in magLims.items():
    print(f"{filter}: {magLim:.2f} +/- {depth[filter].mag_limits.std()<<u.ABmag:.2f} (image {N_SIGMA:.1f}-sigma depth)")

In [ ]:
for filter, d in depth.items():
    with open(DEP_DIR / f"{filter}-DepthApers.reg", "w") as fid:
        fid.write("# Region file format: DS9 version 4.1\n")
        fid.write(f"global color=cyan width=2\n")
        fid.write("image\n")
        for aper in d.apertures[0]:
            x, y = aper.positions
            r = aper.r
            fid.write(f"circle({x:.4f},{y:.4f},{r:.2f})\n")